# Objectives

This work was developed as part of the master's thesis titled  *"The Use of Vector Databases in Predictive Analysis Systems"*.

The notebook presents a machine learning classification system designed for medical diagnosis prediction. The implemented model analyzes clinical data to identify and predict specific disease patterns.

MIMIC-IV (version 2.2) dataset was used as the training material for development and trainig of the model. It comprises:
-   Laboratory test results
    
-   Clinical event records
    
-   Detailed patient admission information


# Imports

In [1]:
# Core data handling
import pandas as pd
import numpy as np

# Machine learning
import xgboost as xgb
import joblib

# sckikit learn components
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score, precision_score


The **'df_balanced_full.csv'** dataset was created through a series of transformations applied to multiple `.csv.gz` files from the MIMIC-IV dataset.

**Data profiling and Merge**

The following files were selected as a result of data profiling. Afterwards, a series of `merge` operations were performed to compile all the information into a single dataset:

1.  `patients.csv.gz`- Demographics

2.  `diagnoses_icd.csv.gz` - ICD diagnosis codes

3.  `chartevents.csv.gz` - Clinical measurements (blood pressure, etc.)

4.  `labevents.csv.gz` - Laboratory test results

5.  `admissions.csv.gz` - Patient admission records

6.  `d_items.csv.gz` - Metadata for chartevents

**Target Selection**

Since the final merge of all the tables resulted in an extremely large dataset. To manage this volume, we focused exclusively on predicting the six most frequent ICD diagnosis codes:

1.  `2724`: Other and unspecified hyperlipidemia

2.  `4019`: Unspecified essential hypertension

3.  `E785`: Hyperlipidemia, unspecified

4.  `Z87891`: Personal history of nicotine dependence

5.  `E039`: Hypothyroidism, unspecified

6.  `Z794`: Long term (current) use of insulin

**Data Cleaning**

-  **Categorical variables**: All missing values were replaced with "Unknown" to maintain dataset consistency while preserving the missing data indicator.

-  **Numerical variables**: NaN values were retained to ensure medical data accuracy, as improper imputation could lead to clinically misleading results.

After processing, the cleaned dataset represents 3% of the original raw data volume. The final dataset contains no missing values in numerical variables and
all ICD code categories were carefully balanced, with each diagnosis containing exactly the same number of cases.

**Variables (Columns)**

 -  **`subject_id`** (Integer) - Unique identifier for each patient, used to link records across tables.

 -  **`value_chartevent`** (String) - Raw clinical observation and measurements.

 -  **`valuenum_chartevent`** (Float) - Numeric representation of clinical measurements.

 -  **`valueuom_chartevent`** (String) - Unit of measurement for chart events.

 -  **`label_chartevent`** (String) - Clinical descriptor of the measurement (e.g., "Heart Rate", "Oxygen Saturation").

 -  **`category`** (String) - High-level classification of event type (e.g., "Vital Signs", "Alarms"). 

 -  **`time_since_admission_chartevent`** (Float) - Hours since admission.

 -  **`icd_code`** (String) - ICD-10 or ICD-9 diagnosis code.

 -  **`admission_type`** (String) - Admission urgency (e.g.,  'Elective', 'Observation admit')

 -  **`insurance`** (String) - Primary payment method (e.g., "Medicare", "Private").

 -  **`race`** (String) - Patient reported ethnicity.

 -  **`admission_location`** (String) - Admission source (e.g., "Emergency Room").

 -  **`age`** (Integer) - Patient age.

 -  **`value_labevent`** (String) - Raw laboratory results.

 -  **`valuenum_labevent`** (Float) - Numeric lab values.

 -  **`valueuom_labevent`** (String) - Laboratory units of measure (e.g., "g/dL", "mg/L").

 -  **`priority`** (String) - Lab test urgency designation (e.g., "ROUTINE", "STAT")

 -  **`time_since_admission_labevent`** (Float) - Hours from admission.

In [2]:
df = pd.read_csv('df_balanced_full.csv', low_memory=False)
display(df)

,subject_id,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,10014078,103,103.0,°F,Temperature Fahrenheit,Routine Vital Signs,4.850000,2724,EW EMER.,Medicaid,UNABLE TO OBTAIN,EMERGENCY ROOM,60,-3,-3.0,mEq/L,Unknown,12.316667
1,10011398,25,25.0,mEq/L,TCO2 (calc) Arterial,Labs,7.900000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,1.4,1.4,Unknown,STAT,4.150000
2,10011398,25,25.0,mA,Temporary Ventricular Stim Setting mA,Cardiovascular (Pacer Data),16.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,384,384.0,mm Hg,Unknown,5.616667
3,10011398,36.8,36.8,°C,Temperature Celsius,Routine Vital Signs,18.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,98,98.0,%,Unknown,6.666667
4,10023771,100,100.0,%,O2 saturation pulseoxymetry,Respiratory,22.750000,2724,ELECTIVE,Medicare,WHITE,PHYSICIAN REFERRAL,70,1.2,1.2,Unknown,ROUTINE,15.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49249,10023117,104,104.0,mg/dL,Glucose (serum),Labs,9.050000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,EMERGENCY ROOM,58,2,2.0,Unknown,STAT,2.666667
49250,10019003,Obeys Commands,6.0,Unknown,GCS - Motor Response,Neurological,1.600000,Z87891,EW EMER.,Other,WHITE,TRANSFER FROM HOSPITAL,72,1.1,1.1,mg/dL,STAT,6.000000
49251,10023117,No response,1.0,Unknown,GCS - Motor Response,Neurological,4.950000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,___,133.0,mEq/L,STAT,2.566667
49252,10037861,No movement,0.0,Unknown,Strength L Arm,Neurological,23.433333,Z87891,EW EMER.,Medicare,UNKNOWN,EMERGENCY ROOM,79,21,21.0,mg/dL,STAT,9.183333


# Treatment - Pipeline

Since the dataset displayed multiple data types (strings, floats, and integers) across its variables, to address the heterogeneous nature it was developed a preprocessing pipeline.

### Data Type Standardization - Categorical Variables**

While XGBoost can handle string variables directly, this approach may result in reduction of the models accuracy. Therefore, all features were transformed into integer and float formats using encoding strategies tailored to each variable's cardinality:

|Carinality|Encoding Method | Variable |
|--|--|--|
|Low|One-Hot Encoding |admission_type  |
|| |insurance  |
|| |priority  |
|Medium|Label Encoding |race |
|| |value_chartevent  |
|| |label_chartevent  |
|| |valueuom_labevent  |
|| |valueuom_chartevent  |
|| |admission_location  |
|| |category  |
|High|Frequency Encoding |value_labevent |
|| |combined (label_chartevent + unit combos) |

### Feature Engineering
**Combined**
To preserve the relationship between clinical measurements and their units, a new variable was created: `combined`.
By merging the `valueuom_chartevent`and `label_chartevent` variables.

**Urgency Scoring System**
An ordinal scoring system was implemented to quantify admission urgency, mirroring hospital emergency protocols:

    urgency_rank = {
        'ELECTIVE': 1,                   # Scheduled procedures
        'OBSERVATION ADMIT': 2,          # Monitoring cases
        'SURGICAL SAME DAY ADMISSION': 3, # Ambulatory surgery
        'URGENT': 4,                     # Acute but not life-threatening
        'DIRECT EMER.': 5,               # Emergency department referrals
        'EW EMER.': 6                    # Life-threatening emergencies
    }

**Feature Interactions**
The urgency rank allowed to create two main feature interactions: 

     # Time-criticality interaction
    df['urgency_x_lab_delay'] = df['urgency_score'] * df['time_since_admission_labevent']
    
    # Age-adjusted urgency
    df['admission_x_age'] = df['urgency_score'] * df['age']

1. **Urgency × Lab Delay Time** - Quantifies the criticality of diagnostic timing
2. **Urgency × Age** - Captures high-risk patients by combinig the age factor

In [3]:
df = df.copy()

In [4]:
class Treatment(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        df = X.copy()
        self.variables_cat = ['admission_type', 'insurance', 'priority']
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.encoder.fit(df[self.variables_cat])
        self.ohe_columns = self.encoder.get_feature_names_out(self.variables_cat)

        self.label_encoders = {}
        variables_label = ['value_chartevent', 'label_chartevent', 'valueuom_labevent',
                           'valueuom_chartevent', 'admission_location', 'race', 'category']
        
        for col in variables_label:
            if col in df.columns:
                le = LabelEncoder()
                le.fit(df[col].astype(str))
                self.label_encoders[col] = le

        return self

    def transform(self, X):
        df = X.copy()

        # Combined
        df['combined'] = np.where(
            (df['valueuom_chartevent'] == 'Unknown'),
            df['label_chartevent'],
            df['label_chartevent'] + ' (' + df['valueuom_chartevent'] + ')'
        )

        # Feature Interactions
        urgency_rank = {
            'ELECTIVE': 1, 'OBSERVATION ADMIT': 2,
            'SURGICAL SAME DAY ADMISSION': 3, 'URGENT': 4,
            'DIRECT EMER.': 5, 'EW EMER.': 6
        }
        df['urgency_score'] = df['admission_type'].map(urgency_rank)
        df['urgency_x_lab_delay'] = df['urgency_score'] * df['time_since_admission_labevent']
        df['admission_x_age'] = df['urgency_score'] * df['age']

        # OneHotEncoding
        one_hot_encoded = self.encoder.transform(df[self.variables_cat])
        one_hot_df = pd.DataFrame(one_hot_encoded, columns=self.ohe_columns, index=df.index)
        df = pd.concat([df.drop(columns=self.variables_cat), one_hot_df], axis=1)

        # LabelEncoding 
        for col, le in self.label_encoders.items():
            if col in df.columns:
                df[col] = le.transform(df[col].astype(str))
            else:
                print(f"Aviso: Coluna '{col}' não encontrada em transform().")

        # Frequency
        df['value_labevent'] = df['value_labevent'].astype(str)
        df['value_labevent_freq'] = df['value_labevent'].map(df['value_labevent'].value_counts())
        df.drop(columns=['value_labevent'], inplace=True)

        df['combined'] = df['combined'].astype(str)
        df['combined_freq'] = df['combined'].map(df['combined'].value_counts())
        df.drop(columns=['combined'], inplace=True)

        return df

# Insert Data in FAISS

In [5]:
import faiss

# Model

**Why the XGBoost? XGBClassifier**

The XGBoost algorithm was chosen for this project due to its strong performance on structured medical data and its frequent use in diagnostic systems. After testing multiple models, including Random Forest and Support Vector Machine (SVM), XGBoost achieved the highest accuracy.

The `XGBClassifier` was selected specifically because this task involves classification rather than regression, which makes a classification model more appropriate for the problem. Additionally, it provides feature importance scores, which are valuable for clinical interpretability.

For hyperparameter tuning, **GridSearchCV** was preferred over **RandomizedSearchCV**, as it yielded slightly better accuracy in our tests, despite the higher computational cost.

### Target Encoding (ICD codes)

In [6]:

le = LabelEncoder()
y = le.fit_transform(df['icd_code'])
X = df.drop('icd_code', axis=1)

In [7]:
print(le.classes_)

['2724' '4019' 'E039' 'E785' 'Z794' 'Z87891']


In [8]:
treatment = Treatment()
X_processed = treatment.fit_transform(X)
X_processed.shape  # (n_samples, n_features)

(49254, 30)

In [9]:
import faiss

index = faiss.IndexFlatL2(30)

In [10]:
index.is_trained

True

In [11]:
index.add(X_processed.astype('float32'))

In [12]:
index.ntotal

49254

### Train-Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, stratify=y, test_size=0.2, random_state=42)

### Optimized XGBoost configuration and hyperparameters

In [14]:

param_dist = {
    'model__max_depth': [6],
    'model__learning_rate': [0.1],
    'model__gamma': [0.5],
    'model__min_child_weight': [3],
    'model__reg_alpha': [0],
    'model__reg_lambda': [1],
    'model__n_estimators': [200]  
}


pipeline = Pipeline([
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(le.classes_),
        tree_method='hist',
        eval_metric='mlogloss',
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        random_state=42
    ))
])

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_dist,
    scoring='precision_weighted',
    cv=5,
    n_jobs=-1,
    verbose=2 )

### Training Process

In [15]:

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\catar\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [15:54:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=0.8,
                                                      device=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='mlogloss',
                                                      feature_types=None,
                                                      feature_weights=None,
                                                      gamma=None,
                                                      grow_policy=None,
                                                      importance_type=No...
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None, num_class=6, ...))]),
             n_jobs=-1,
             param_grid={'model__gamma': [0.5], 'model__learning_rate': [0.1],
                         'model__max_depth': [6],
                         'model__min_child_weight': [3],
                         'model__n_estimators': [200], 'model__reg_alpha': [0],
                         'model__reg_lambda': [1]},
             scoring='precision_weighted', verbose=2)

In [16]:

y_pred = grid_search.predict(X_test)

# mean CV = 5
print("Accuracy - Internal validation:", grid_search.best_score_)

print("\nAccuracy - Real", accuracy_score(y_test, y_pred))
print("Precision (macro):", precision_score(y_test, y_pred, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy - Internal validation: 0.6447120561723713

Accuracy - Real 0.6503908232666734
Precision (macro): 0.6521365227851583

Classification Report:
               precision    recall  f1-score   support

        2724       0.68      0.82      0.74      1641
        4019       0.77      0.62      0.69      1642
        E039       0.62      0.57      0.59      1642
        E785       0.61      0.25      0.35      1642
        Z794       0.72      0.98      0.83      1642
      Z87891       0.51      0.67      0.58      1642

    accuracy                           0.65      9851
   macro avg       0.65      0.65      0.63      9851
weighted avg       0.65      0.65      0.63      9851



### Model saving

In [17]:
joblib.dump(le, 'label_encoder_icd.pkl')           # Saves diagnosis mapping
joblib.dump(treatment, 'treatment.pkl')            # Saves preprocessing 
joblib.dump(grid_search.best_estimator_, 'modelo_pipeline_final.pkl')  # Full pipeline
faiss.write_index(index, 'faiss_index.bin')

# Example

## Download 

In [18]:
treatment = joblib.load('treatment.pkl')
pipeline = joblib.load('modelo_pipeline_final.pkl')
le = joblib.load('label_encoder_icd.pkl')
faiss_index = faiss.read_index('faiss_index.bin')

In [19]:
faiss_index = faiss.read_index('faiss_index.bin')

In [20]:
cases = pd.read_csv('cases.csv')

In [21]:
case = treatment.transform(cases).astype("float32")
expected_cols = pipeline.named_steps['model'].feature_names_in_
missing_cols = set(expected_cols) - set(case.columns)

print("Missing columns:")
print(missing_cols)


Missing columns:
set()


In [22]:
display(cases)

,subject_id,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,1,No movement,36.8,Unknown,Temperature Celsius,Routine Vital Signs,12.00,URGENT,Medicare,WHITE,EMERGENCY ROOM,72,2.0,2.0,mg/dL,STAT,1.20
1,2,27,27.0,mA,Temporary Ventricular Stim Setting mA,Cardiovascular (Pacer Data),2.50,EW EMER.,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,333.0,333.0,mm Hg,Unknown,2.00
2,3,8,8.0,insp/min,Resp Alarm - Low,Alarms,12.55,EW EMER.,Other,WHITE,PHYSICIAN REFERRAL,59,8.9,8.9,g/dL,STAT,14.95


# Results

Three cases were created as a way to test the model. The first two are synthetic (non-real) cases used solely to test how the model classifies them. The third case, however, is a real example taken from the training dataset.

The model predicted **Unspecified essential hypertension** for the first two synthetic cases, and **Long-term (current) use of insulin** for the real case.

This suggests that the model performs reasonably well, as it correctly predicted the diagnosis of the real case from the training dataset: Long-term (current) use of insulin.

However, the model still needs to improve its precision.

In [23]:
preds = pipeline.predict(case)
print(preds)

[1 1 4]


In [24]:
lable = pd.read_csv('..\models\icd_match.csv')

In [25]:
icd_dict = dict(zip(lable['icd_code'], lable['long_title']))

In [26]:

predicted_icd_codes = le.inverse_transform(preds)
for i, (index, row) in enumerate(cases.iterrows()):
    icd_code = predicted_icd_codes[i]
    description = icd_dict.get(icd_code)
    
    print(f"Example {i+1}:")
    print(row.to_dict())  
    print(f"Result: {icd_code} - {description}")
    print("-" * 40)


Example 1:
{'subject_id': 1, 'value_chartevent': 'No movement', 'valuenum_chartevent': 36.8, 'valueuom_chartevent': 'Unknown', 'label_chartevent': 'Temperature Celsius', 'category': 'Routine Vital Signs', 'time_since_admission_chartevent': 12.0, 'admission_type': 'URGENT', 'insurance': 'Medicare', 'race': 'WHITE', 'admission_location': 'EMERGENCY ROOM', 'age': 72, 'value_labevent': 2.0, 'valuenum_labevent': 2.0, 'valueuom_labevent': 'mg/dL', 'priority': 'STAT', 'time_since_admission_labevent': 1.2}
Result: 4019 - Unspecified essential hypertension
----------------------------------------
Example 2:
{'subject_id': 2, 'value_chartevent': '27', 'valuenum_chartevent': 27.0, 'valueuom_chartevent': 'mA', 'label_chartevent': 'Temporary Ventricular Stim Setting mA', 'category': 'Cardiovascular (Pacer Data)', 'time_since_admission_chartevent': 2.5, 'admission_type': 'EW EMER.', 'insurance': 'Other', 'race': 'HISPANIC OR LATINO', 'admission_location': 'PHYSICIAN REFERRAL', 'age': 67, 'value_labe

In [36]:
print(case.columns.equals(X_train.columns))  # deve ser True


True


In [37]:
print("X_train summary:")
print(pd.DataFrame(X_train).describe())

print("\ncase summary:")
print(pd.DataFrame(case).describe())


X_train summary:
         subject_id  value_chartevent  valuenum_chartevent  \
count  3.940300e+04      39403.000000         39403.000000   
mean   1.002332e+07        543.672842            64.674516   
std    9.720324e+03        378.705051           692.162727   
min    1.000122e+07          0.000000           -23.000000   
25%    1.001808e+07        149.000000             1.800000   
50%    1.002312e+07        573.000000            18.000000   
75%    1.002929e+07        914.000000            83.000000   
max    1.004002e+07       1076.000000         62656.000000   

       valueuom_chartevent  label_chartevent      category  \
count         39403.000000      39403.000000  39403.000000   
mean             11.328249        175.023602     11.881633   
std               9.099376         94.387330      5.435587   
min               0.000000          0.000000      0.000000   
25%               5.000000         97.000000     10.000000   
50%               6.000000        198.000000     13.

In [34]:
if len(case.shape) == 1:
    novo_tratado = case.reshape(1, -1)

D, I = faiss_index.search(case, k=3)
print("Distâncias:", D)
print("Índices mais semelhantes:", I)

Distâncias: [[1.00024319e+14 1.00024319e+14 1.00024319e+14]
 [1.00024302e+14 1.00024302e+14 1.00024302e+14]
 [1.00024285e+14 1.00024285e+14 1.00024285e+14]]
Índices mais semelhantes: [[8280 8669 9212]
 [8280 8669 9212]
 [8280 8669 9212]]


In [35]:
for i, (index, dist) in enumerate(zip(I, D)):
    print(f"\n🔍 Similar case {i + 1}:\n")
    
    # Cria um dataframe temporário só com os casos semelhantes
    similares = df.iloc[index].copy()
    similares["dist"] = dist

    print(similares.to_string(index=False))


🔍 Similar case 1:

 subject_id value_chartevent  valuenum_chartevent valueuom_chartevent                        label_chartevent     category  time_since_admission_chartevent icd_code admission_type insurance  race admission_location  age value_labevent  valuenum_labevent valueuom_labevent priority  time_since_admission_labevent         dist
   10001217   Obeys Commands                  6.0             Unknown                    GCS - Motor Response Neurological                        23.666667     4019   DIRECT EMER.     Other WHITE PHYSICIAN REFERRAL   55              2                2.0             #/hpf  ROUTINE                      17.450000 1.000243e+14
   10001217               92                 92.0                   % O2 Saturation Pulseoxymetry Alarm - Low       Alarms                        23.033333     4019   DIRECT EMER.     Other WHITE PHYSICIAN REFERRAL   55            9.8                9.8                 %  ROUTINE                      11.866667 1.000243e+14
   10

# IVF - Verenoi Cells

In [ ]:
nlist = 50
quantizer = faiss_index.IndexFlatL2(30)
index = faiss.IndexIVFFlat(quantizer, 30, nlist)

# Product quatization

# Locallity Sensitive Hashing

# HSNW